# Операции с файлом базы данных

Инициализация рабочей директории и настроек

In [1]:
import os
import sys
from pathlib import Path

def init_work_dir(path):
    # Установить рабочую директорию
    project_root = Path(path)
    os.chdir(project_root)
    # Добавить путь к модулям проекта
    sys.path.insert(0, str(project_root))
    print(f"Текущая рабочая директория: {os.getcwd()}")

init_work_dir("C:/Projects/GeoOffice")

from src.utils.file_utils import FileUtils
from src.models.settings_model import Settings

settings = Settings(data=None)
settings_data = FileUtils.load_json(Path("storage") / "data" / "settings.json")
try:
    settings = Settings(data=settings_data)
    print("✅ Настройки загружены успешно")
except Exception as e:
    print(f"Ошибка чтения файла настроек: {e}")
    print("Используются настройки по умолчанию")

DB_PATH = settings.paths.database_path
print(f"База данных: {DB_PATH}")

Текущая рабочая директория: C:\Projects\GeoOffice
14:10:29 | INFO     | GeoOffice | ============================================================
14:10:29 | INFO     | GeoOffice | 🚀 Запуск приложения GeoOffice
14:10:29 | INFO     | GeoOffice | 📅 Дата и время: 2025-07-20 14:10:29
14:10:29 | INFO     | GeoOffice | 🐍 Версия Python: 3.13.5 (tags/v3.13.5:6cb20a2, Jun 11 2025, 16:15:46) [MSC v.1943 64 bit (AMD64)]
14:10:29 | INFO     | GeoOffice | 📁 Рабочая директория: C:\Projects\GeoOffice
14:10:29 | INFO     | GeoOffice | 📂 Директория логов: C:\Projects\GeoOffice\storage\logs
14:10:29 | INFO     | GeoOffice | ============================================================
14:10:29 | DEBUG    | GeoOffice.utils.files | 📂 Загрузка JSON файла: storage\data\settings.json
14:10:29 | INFO     | GeoOffice.utils.files | ✅ JSON файл загружен: storage\data\settings.json
✅ Настройки загружены успешно
База данных: C:\GeoOffice_test_server\geo_office.db


Установка / снятие защиты

In [3]:
import subprocess

def show_db_file():
    subprocess.run(['attrib', '-h', '-s', str(DB_PATH)], check=True)
def protect_db_file():
    subprocess.run(['attrib', '+h', '+s', str(DB_PATH)], check=True)

show_db_file()  # Снятие защиты
# protect_db_file()  # Установка защиты

Удаление БД

In [4]:
from pathlib import Path

def delete_db_file():
    Path(DB_PATH).unlink()
    if Path(DB_PATH).exists():
        raise Exception("Файл не удалён")

# delete_db_file()

Создание / Подключение к БД

In [15]:
from src.services.database_service import DatabaseService

db_service = DatabaseService(DB_PATH)

14:23:36 | INFO     | GeoOffice.services.database_service | Инициализация базы данных: C:\GeoOffice_test_server\geo_office.db
14:23:36 | INFO     | GeoOffice.services.database_service | База данных успешно инициализирована


# Заполнение базы данных

Заполнение БД тестовыми данными

In [12]:
test_projects = [
    {
        'number': '201.17',
        'name': 'Инженерные сети и благоустройство жилого района усадебной застройки «Сосны» в г.Миоры',
        'customer': 'Дочернее коммунальное унитарное предприятие «Управление капитального строительства Глубокского района»',
        'chief_engineer': 'Ладисова Татьяна Геннадьевна',
        'status': 'активный',
        'address': 'Витебская область, Миорский район, г.Миоры',
        'path': '2025\\Миорский район\\201.17 Сосны, Миоры'
    },
    {
        'number': '92.25',
        'name': 'Возведение многоквартирного жилого дома по ул. Гагарина в г. Городоке',
        'customer': 'Государственное предприятие «УКС Городокского района»',
        'chief_engineer': 'Ладисова Татьяна Геннадьевна',
        'status': 'активный',
        'address': 'Витебская область, г. Городок, ул. Гагарина',
        'path': '2025\\Городокский район\\92.25 ЖД Городок ул.Гагарина'
    },
    ]

In [6]:
for test_project in test_projects:
    db_service.create_project(**test_project)

16:10:22 | DEBUG    | GeoOffice.services.database_service | Создан новый проект: Project(id=None, number='201.17', name='Инженерные сети и благоустройство жилого района усадебной застройки «Сосны» в г.Миоры')
16:10:22 | DEBUG    | GeoOffice.services.database_service | Создан новый проект: Project(id=None, number='92.25', name='Возведение многоквартирного жилого дома по ул. Гагарина в г. Городоке')


# Просмотр объектов

In [22]:
from datetime import datetime
from src.models.project_model import Project

project_data = db_service.get_project(1).to_dict()
project_data['created_date'] = datetime.fromisoformat(project_data['created_date'])
project_data['modified_date'] = datetime.fromisoformat(project_data['modified_date'])
Project(**project_data)

20:59:19 | DEBUG    | GeoOffice.services.database_service | Получение проекта по id: id=1


Project(id=1, number='50.25', name='Реконструкция спортзала по адресу ул.Молодежная 26 в г.Новополоцк', customer='Новополоцкий городской исполнительный комитет', chief_engineer='Богданова Екатерина Александровна', status='активный', address='ул.Молодежная 26 в г.Новополоцк', path='2025\\Новополоцк\\50.25 Реконструкция спортзала Новополоцк', created_date=datetime.datetime(2025, 7, 5, 20, 30, 22, 952641), modified_date=datetime.datetime(2025, 7, 5, 20, 30, 22, 952649))

In [9]:
all_projects = db_service.get_all_projects()
for project in all_projects:
    print(project.number, project.name)

17:07:40 | DEBUG    | GeoOffice.services.database_service | Получение всех проектов
201.17 Инженерные сети и благоустройство жилого района усадебной застройки «Сосны» в г.Миоры
92.25 Возведение многоквартирного жилого дома по ул. Гагарина в г. Городоке


# Прочие операции

In [11]:
all_projects = db_service.get_all_projects()
for project in all_projects:
    print(project.path)

17:10:01 | DEBUG    | GeoOffice.services.database_service | Получение всех проектов
2025\Миорский район\201.17 Сосны, Миоры
2025\Городокский район\92.25 ЖД Городок ул.Гагарина


In [16]:
db_service.get_all_projects()

18:40:20 | DEBUG    | GeoOffice.services.database_service | Получение всех проектов


[ProjectTable[1], ProjectTable[2]]

In [50]:
from pprint import pprint
import re
import os
from pathlib import Path

project_dirname_pattern = re.compile(r"^(?:\d{1,3}|NN)\.\d{2}(?:\s.+)?$")

projects_dirpath = Path(settings.paths.file_server) / settings.paths.projects_folder

projects_in_files = []
for dirpath, dirnames, filenames in os.walk(projects_dirpath):
    for dirname in dirnames:
        if project_dirname_pattern.match(dirname):
            match = re.match(r"^((?:\d{1,3}|NN)\.\d{2})(?:\s+(.*))?$", dirname)
            number = match.group(1)
            name = match.group(2) or ""
            projects_in_files.append(tuple([Path(dirpath) / dirname, number, name]))

pprint(projects_in_files)

projects_in_files_rel = set(str(p.relative_to(projects_dirpath)) for p, _, _ in projects_in_files)
projects_in_db = list(db_service.get_all_projects())
db_paths = set(p.path for p in projects_in_db)

result = {
    "only_in_disk": list(projects_in_files_rel - db_paths),
    "only_in_db": list(db_paths - projects_in_files_rel),
    "in_both": list(projects_in_files_rel & db_paths)
}
pprint(result)

[(WindowsPath('C:/GeoOffice_test_server/Work/Объекты/2025/Миорский район/201.17 Сосны, Миоры'),
  '201.17',
  'Сосны, Миоры'),
 (WindowsPath('C:/GeoOffice_test_server/Work/Объекты/2025/Новополоцк/50.25 Реконструкция спортзала Новополоцк'),
  '50.25',
  'Реконструкция спортзала Новополоцк'),
 (WindowsPath('C:/GeoOffice_test_server/Work/Объекты/2025/Ушачский район/116.25 Ушачи вынос осей'),
  '116.25',
  'Ушачи вынос осей'),
 (WindowsPath('C:/GeoOffice_test_server/Work/Объекты/2025/Ушачский район/172.25 ЖД Ушачи'),
  '172.25',
  'ЖД Ушачи'),
 (WindowsPath('C:/GeoOffice_test_server/Work/Объекты/2025/Шарковщинский район/72.25 ЖД Шарковщина'),
  '72.25',
  'ЖД Шарковщина')]
23:35:41 | DEBUG    | GeoOffice.services.database_service | Получение всех проектов
{'in_both': ['2025\\Миорский район\\201.17 Сосны, Миоры'],
 'only_in_db': ['2025\\Городокский район\\92.25 ЖД Городок ул.Гагарина'],
 'only_in_disk': ['2025\\Новополоцк\\50.25 Реконструкция спортзала Новополоцк',
                  '2025\\

In [47]:
type(print)

builtin_function_or_method

Сбор данных объектов в базу данных

In [24]:
import os
projects = []
for dirpath, dirnames, filenames in os.walk(
        Path(settings.paths.file_server) / settings.paths.projects_folder
):
    for filename in filenames:
        if filename == '0.txt' or filename == '0.1.txt' or filename == '0.2.txt':
            path = Path(dirpath).relative_to(Path(settings.paths.file_server) / settings.paths.projects_folder)
            with open(Path(dirpath) / filename, 'r', encoding='utf8') as f:
                number, name, customer = f.readlines()
                projects.append(
                    {'number': number.strip(),
                     'name': name.strip(),
                     'customer': customer.strip(),
                     'path': str(path)}
                )

In [23]:
for project in projects:
    db_service.create_project(
        **project,
        chief_engineer='не указан',
        status='не указан',
        address='не указан'
    )

14:41:23 | DEBUG    | GeoOffice.services.database_service | Создан новый проект: Project(id=None, number='92.25', name='ЖД Городок ул.Гагарина')
14:41:23 | DEBUG    | GeoOffice.services.database_service | Создан новый проект: Project(id=None, number='201.17', name='Сосны, Миоры')
14:41:23 | DEBUG    | GeoOffice.services.database_service | Создан новый проект: Project(id=None, number='50.25', name='Реконструкция спортзала Новополоцк')
14:41:23 | DEBUG    | GeoOffice.services.database_service | Создан новый проект: Project(id=None, number='116.25', name='Ушачи вынос осей')
14:41:23 | DEBUG    | GeoOffice.services.database_service | Создан новый проект: Project(id=None, number='172.25', name='ЖД Ушачи')
14:41:23 | DEBUG    | GeoOffice.services.database_service | Создан новый проект: Project(id=None, number='72.25', name='ЖД Шарковщина')
